In [68]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
import tensorflow_hub as hub
import cv2
import shutil
import random

In [78]:
# Set some paths
# Download images to a folder in this directory and set PATH variable to its path
PATH = "/Users/jeremystubbs/Desktop/Python/Machine_Learning_NNs/Mobilenet/mobilenet_custom/archive"
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'valid')
images_dir = os.path.join(PATH, "images.csv")

if os.path.isdir(train_dir) is False:
    os.mkdir(train_dir)
if os.path.isdir(validation_dir) is False:
    os.mkdir(validation_dir)

# Create list of all unique labels from the csv file
with open(images_dir) as f:
    all_image_paths = [row.split(",")[0] for row in f]
with open(images_dir) as f:
    all_image_labels = [row.split(",")[2] for row in f] 

unique_labels = set(all_image_labels)

# Make subfolders for each class
for item in unique_labels:
  if os.path.isdir('train/'+str(item)) is False:
    os.mkdir('train/'+str(item))
  if os.path.isdir('valid/'+str(item)) is False:
    os.mkdir('valid/'+str(item))

In [80]:
# Move all pictures to images_folder subfolders - change second part to your folder name
IMAGES_PATH = os.path.join(PATH, 'images_original')
for item in unique_labels:
    # print(item)
    temp_arr = []
    if item == "label":
        continue
    for i in range(len(all_image_labels)):
        if all_image_labels[i]==item:
            temp_arr.append(all_image_paths[i])

    k = int(len(temp_arr) * 0.8)
    train_temp_arr = random.sample(temp_arr, k)  
    valid_temp_arr = list(set(temp_arr).difference(train_temp_arr))
    # print(len(train_temp_arr), len(valid_temp_arr))
    # print(IMAGES_PATH+train_temp_arr[0]+".jpg")
    # print(valid_temp_arr[0])

    for val in train_temp_arr:
        try:
            shutil.copy(IMAGES_PATH +"/"+ val+".jpg", train_dir+"/"+item)
        except:
            x=2

    for each in valid_temp_arr:
        try:
            shutil.copy(IMAGES_PATH +"/"+ each+".jpg", validation_dir+"/"+item)
        except:
            x=2


Undershirt
94 24
/Users/jeremystubbs/Desktop/Python/Machine_Learning_NNs/Mobilenet/mobilenet_custom/archive/images_originalf116a69b-3544-4ffd-8d7b-116def60fdab.jpg
62384b64-25d5-4ed0-9fc2-d05c4af0b894
label
T-Shirt
808 203
/Users/jeremystubbs/Desktop/Python/Machine_Learning_NNs/Mobilenet/mobilenet_custom/archive/images_originala526cbda-d3c9-467a-8090-98ce0b28dcdd.jpg
7fe4daa4-2802-42b4-8dee-68c114105a7a
Other
53 14
/Users/jeremystubbs/Desktop/Python/Machine_Learning_NNs/Mobilenet/mobilenet_custom/archive/images_original6e69b257-4eaf-4785-95a9-f984ee195f20.jpg
f88a9c13-fd71-42e6-9ea6-8b0597f1cce1
Polo
96 24
/Users/jeremystubbs/Desktop/Python/Machine_Learning_NNs/Mobilenet/mobilenet_custom/archive/images_original7dd9bded-33f2-4928-b8f1-7efe0fd3bce3.jpg
bdb96979-3a9a-4d64-98e6-bda684e7e81e
Outwear
249 63
/Users/jeremystubbs/Desktop/Python/Machine_Learning_NNs/Mobilenet/mobilenet_custom/archive/images_original7183e0de-c9e1-45bf-9d89-9319afc727ec.jpg
efbc5815-d1d4-4d85-8df2-9d9caae4300b
Dre

In [81]:
# Make training and validation Datasets
BATCH_SIZE = 10
IMG_SIZE = (224, 224)

train_dataset = tf.keras.utils.image_dataset_from_directory(train_dir,
                                                            labels = "inferred",
                                                            shuffle=True,
                                                            batch_size=BATCH_SIZE,
                                                            image_size=IMG_SIZE)

validation_dataset = tf.keras.utils.image_dataset_from_directory(validation_dir,
                                                                labels = "inferred",
                                                                shuffle=True,
                                                                batch_size=BATCH_SIZE,
                                                                image_size=IMG_SIZE)

class_names = train_dataset.class_names


Found 4314 files belonging to 21 classes.
Found 1089 files belonging to 21 classes.


In [ ]:
# Plot some images
plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [106]:
# Define preprocessing function and create train and validation iterators

preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

# This does the same thing
# rescale = tf.keras.layers.Rescaling(1./127.5, offset=-1)

# Could include preprocessing here
# train_dataset = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input).flow_from_directory(directory=train_dir, target_size=(224,224), batch_size=10)
# validation_dataset = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input).flow_from_directory(directory=validation_dir,target_size=(224,224), batch_size=10)

train_dataset = tf.keras.preprocessing.image.ImageDataGenerator().flow_from_directory(directory=train_dir, target_size=(224,224), batch_size=10)
validation_dataset = tf.keras.preprocessing.image.ImageDataGenerator().flow_from_directory(directory=validation_dir,target_size=(224,224), batch_size=10)

image_batch, label_batch = next(iter(train_dataset))
# print(image_batch, label_batch)
# print(type(train_dataset))

Found 3714 images belonging to 13 classes.
Found 935 images belonging to 13 classes.


In [107]:
# Create the base model from the pre-trained model MobileNet V2
IMG_SHAPE = IMG_SIZE + (3,)

base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

base_model2 = tf.keras.applications.MobileNetV2()

base_model.trainable = False

# Optional fine-tuning I read on tensorflow's official website
# Let's take a look to see how many layers are in the base model
# print("Number of layers in the base model: ", len(base_model.layers))

# # Fine-tune from this layer onwards
# fine_tune_at = 100

# # Freeze all the layers before the `fine_tune_at` layer
# for layer in base_model.layers[:fine_tune_at]:
#   layer.trainable = False

In [108]:
##This cell creates the model
# Add input, preprocessing, global average and prediction layer to the base model
# feature_batch, and prediction_batch are optional 
# feature_batch = base_model(image_batch)
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
# feature_batch_average = global_average_layer(feature_batch)
# prediction_layer = tf.keras.layers.Dense(len(class_names),
                        #   kernel_regularizer=tf.keras.regularizers.l2(0.0001))
# prediction_batch = prediction_layer(feature_batch_average)
prediction_layer = tf.keras.layers.Dense(units=13, activation='softmax')

inputs = tf.keras.Input(shape=(224, 224, 3))
x = preprocess_input(inputs)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)


In [109]:
##Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Here is an alternative setup
# model.compile(
#   optimizer=tf.keras.optimizers.SGD(learning_rate=0.005, momentum=0.9), 
#   loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.1),
#   metrics=['accuracy'])

# model.summary()



In [110]:
#Train the model
initial_epochs = 10

model.fit(train_dataset,
                    epochs=initial_epochs, 
                    validation_data=validation_dataset, )



Epoch 1/10
372/372 [==============================] - 282s 756ms/step - loss: 1.0827 - accuracy: 0.6605 - val_loss: 0.6938 - val_accuracy: 0.7775
Epoch 2/10
372/372 [==============================] - 280s 752ms/step - loss: 0.5960 - accuracy: 0.8094 - val_loss: 0.5787 - val_accuracy: 0.7979
Epoch 3/10
372/372 [==============================] - 281s 754ms/step - loss: 0.4780 - accuracy: 0.8446 - val_loss: 0.5590 - val_accuracy: 0.8150
Epoch 4/10
372/372 [==============================] - 279s 750ms/step - loss: 0.4081 - accuracy: 0.8654 - val_loss: 0.5450 - val_accuracy: 0.8118
Epoch 5/10
372/372 [==============================] - 279s 750ms/step - loss: 0.3650 - accuracy: 0.8732 - val_loss: 0.5498 - val_accuracy: 0.8139
Epoch 6/10
372/372 [==============================] - 279s 750ms/step - loss: 0.3212 - accuracy: 0.8880 - val_loss: 0.5989 - val_accuracy: 0.8032
Epoch 7/10
372/372 [==============================] - 282s 757ms/step - loss: 0.2953 - accuracy: 0.8998 - val_loss: 0.5827 -

In [114]:
## Predict images in test set with model 
img = cv2.imread("dress2.jpg")
resized = cv2.resize(img, (224, 224))
reshaped = resized.reshape(1, 224, 224, 3)
# print(type(reshaped), reshaped.shape)


predictions = model.predict(reshaped, verbose=0)
print(predictions)
# Note: the label map is ['Blazer','Dress','Hat','Hoodie', 'Longsleeve','Pants', 'Polo','Shirt','Shoes','Shorts', 'Skirt','T-Shirt','Undershirt']

[[7.6554790e-07 9.7837526e-01 7.6489923e-05 1.4933976e-03 6.4658729e-04
  8.7753988e-06 4.0986665e-04 1.8605724e-02 1.2913530e-04 1.3564948e-04
  1.4675536e-05 8.2458348e-05 2.1192469e-05]]


In [ ]:
##Save model
os.chdir("/Users/jeremystubbs/Desktop/Python/Machine_Learning_NNs/Mobilenet/mobilenet_custom/")
!mkdir -p saved_model
model.save('saved_model/my_model')